<a href="https://colab.research.google.com/github/zhukuixi/Udacity_DataScientistNanoDegree/blob/main/MovieTweeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/zhukuixi/Udacity_DataScientistNanoDegree

Cloning into 'Udacity_DataScientistNanoDegree'...
remote: Enumerating objects: 277, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 277 (delta 51), reused 69 (delta 29), pack-reused 177
Receiving objects: 100% (277/277), 135.37 MiB | 19.52 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('/content/Udacity_DataScientistNanoDegree/MovieTweet/data/original_movies.dat',
                     delimiter='::',
                     header=None,
                     names=['movie_id', 'movie', 'genre'],
                     dtype={'movie_id': object}, engine='python')

reviews = pd.read_csv('/content/Udacity_DataScientistNanoDegree/MovieTweet/data/original_ratings.dat',
                      delimiter='::',
                      header=None,
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      dtype={'movie_id': object, 'user_id': object, 'timestamp': object},
                      engine='python')

# Reduce the size reviews dataset
reviews = reviews.loc[:100000,:]

# 1.Data Exploration


In [36]:
print(movies.head())
print(reviews.head())
print(movies.isna().mean())
print(reviews.isna().mean())

  movie_id                                              movie  \
0  0000008      Edison Kinetoscopic Record of a Sneeze (1894)   
1  0000010                La sortie des usines Lumière (1895)   
2  0000012                      The Arrival of a Train (1896)   
3       25  The Oxford and Cambridge University Boat Race ...   
4  0000091                         Le manoir du diable (1896)   

               genre  
0  Documentary|Short  
1  Documentary|Short  
2  Documentary|Short  
3                NaN  
4       Short|Horror  
  user_id  movie_id  rating   timestamp
0       1   0114508       8  1381006850
1       2   0208092       5  1586466072
2       2   0358273       9  1579057827
3       2  10039344       5  1578603053
4       2   6751668       9  1578955697
movie_id    0.000000
movie       0.000000
genre       0.007018
dtype: float64
user_id      0.0
movie_id     0.0
rating       0.0
timestamp    0.0
dtype: float64


In [54]:

dict_sol1 = {
'The number of movies in the dataset': movies.shape[0],
'The number of ratings in the dataset': reviews.shape[0],
'The number of different genres': movies['genre'].str.split("|").to_frame('genre').explode('genre')['genre'].nunique(),
'The number of unique users in the dataset': reviews['user_id'].nunique(),
'The number missing ratings in the reviews dataset': sum(pd.isna(reviews['rating'])),
'The average rating given across all ratings': reviews['rating'].mean(),
'The minimum rating given across all ratings': reviews['rating'].min(),
'The maximum rating given across all ratings': reviews['rating'].max()
}
dict_sol1

{'The number of movies in the dataset': 35479,
 'The number of ratings in the dataset': 100001,
 'The number of different genres': 28,
 'The number of unique users in the dataset': 8022,
 'The number missing ratings in the reviews dataset': 0,
 'The average rating given across all ratings': 7.397666023339767,
 'The minimum rating given across all ratings': 0,
 'The maximum rating given across all ratings': 10}

#2.Data Cleaning

we need to pull some additional relevant information out of the existing columns.  

For each of the datasets, there are a couple of cleaning steps we need to take care of:  

Movies  
- Pull the date from the title and create new column  
- Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)  
- Dummy column the genre with 1's and 0's for each genre  

Reviews  
- Create a date out of time stamp  



In [9]:
movies['date'] = movies['movie'].str[-5:-1]
movies['genre'] = movies['genre'].str.split('|')
dummy_time = pd.get_dummies(movies['date'].str[:2]+"00's")
movies_new = pd.concat([movies,dummy_time],axis=1)

In [10]:
movies_new = movies_new.explode('genre')
dummy_genre = pd.get_dummies(movies_new['genre'])
movies_new = pd.concat([movies,dummy_genre],axis=1)

In [27]:
movies_new.head()

,movie_id,movie,genre,date,Action,Adult,Adventure,Animation,Biography,Comedy,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),"[Documentary, Short]",1894,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),"[Documentary, Short]",1894,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),"[Documentary, Short]",1895,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),"[Documentary, Short]",1895,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),"[Documentary, Short]",1896,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
from datetime import datetime
reviews['date'] = reviews['timestamp'].apply(lambda x:datetime.fromtimestamp(int(x)))
reviews_new = reviews
reviews_new.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-05 21:00:50
1,2,0208092,5,1586466072,2020-04-09 21:01:12
2,2,0358273,9,1579057827,2020-01-15 03:10:27
3,2,10039344,5,1578603053,2020-01-09 20:50:53
4,2,6751668,9,1578955697,2020-01-13 22:48:17
